# Datengenerator und Bewertung
    Szenario der Bewährungsauflagen Bewertung mit 5 Attributen(Name, Hautfarbe, Laufende_Strafe in Jahre, Geschlecht und Härte des Vergehens)
    Dabei können Hautfarbe und Geschlecht durch Bias beeinflusst werden.

In [10]:
import numpy as np
from faker import Faker
import pandas as pd
import os
from datetime import datetime
import random

fake = Faker()

In [11]:
def erstelle_fake_data(num = 10, seed = 123):
    np.random.seed(seed)
    fake.seed_instance(seed)
    output = []
    for x in range(num):
        geschlecht = np.random.choice(["M", "W"], p=[0.80, 0.20])
        haerte = np.random.choice(["Leicht", "Mittel", "Hart"], p=[0.45, 0.35, 0.2]) if geschlecht=="M" else np.random.choice(["Leicht", "Mittel", "Hart"], p=[0.6, 0.3, 0.1])
        hautfarbe = np.random.choice(["White", "Black"], p=[0.25, 0.75]) if geschlecht=="M" else np.random.choice(["White", "Black"], p=[0.65, 0.35])
        output.append(
            {
                "Name": fake.first_name_male() if geschlecht=="M" else fake.first_name_female(),
                "Hautfarbe": hautfarbe,
                "Laufende_Strafe": np.random.choice([1,2,3,4,5]),
                "Geschlecht": geschlecht,
                "Haerte_des_Vergehens": haerte
            }  
        )
    return output

In [12]:
def regeln_erstellen(antrag_werte, antrag_bias, bias):
    regel_pos = {}
    regel_neg = {}
    regel_neg_bias = {}
    regel_pos_bias = {}
    for key in antrag_werte.keys():
        values = antrag_werte[key]
        length = int(len(values)/2)+1
        pos_values = []
        for x in range(length):
            pos_values.append(values[x])
        regel_pos[key] = pos_values
        neg_values = []
        for y in range(len(values)):
            if(y >= length):
                neg_values.append(values[y])
        regel_neg[key] = neg_values
    regel_pos_bias = regel_pos.copy()
    regel_neg_bias = regel_neg.copy()
    if(bias != None):
        for key in antrag_bias.keys():
            if(bias.keys().__contains__(key)):
                regel_neg_bias[key] = bias[key]
                neg_bias = bias[key]
                regel_pos_bias[key] = []
                for val in antrag_bias[key]:
                    if not(neg_bias.__contains__(val)):
                        regel_pos_bias[key].append(val)
    return(regel_pos, regel_neg, regel_pos_bias, regel_neg_bias)

In [13]:
bias = {
    "Hautfarbe": ["Black"]
}
antrag_werte = {
    "Laufende_Strafe": [1,2,3,4,5],
    "Haerte_des_Vergehens": ["Leicht", "Mittel", "Hart"]
}
antrag_bias = {
    "Hautfarbe": ["Black", "White"],
    "Geschlecht": ["M", "W"]
}
r_pos, r_neg, rb_pos, rb_neg = regeln_erstellen(antrag_werte, antrag_bias, bias)
print(r_pos)
print(r_neg)
print(rb_pos)
print(rb_neg)
print(rb_pos.__len__())

{'Laufende_Strafe': [1, 2, 3], 'Haerte_des_Vergehens': ['Leicht', 'Mittel']}
{'Laufende_Strafe': [4, 5], 'Haerte_des_Vergehens': ['Hart']}
{'Laufende_Strafe': [1, 2, 3], 'Haerte_des_Vergehens': ['Leicht', 'Mittel'], 'Hautfarbe': ['White']}
{'Laufende_Strafe': [4, 5], 'Haerte_des_Vergehens': ['Hart'], 'Hautfarbe': ['Black']}
3


In [14]:
def seed_generieren():
    now = datetime.now()
    seed = (now.day * now.minute * now.second * now.month * now.year * now.hour) / now.microsecond 
    print(seed)
    if(seed <= 0):
        seed = now.day * (now.minute + 1)
    return seed

In [15]:
class Bewerter1:
    def __init__(self, regeln_pos, regeln_neg, bias):
        self.regeln_pos = regeln_pos
        self.regeln_neg = regeln_neg
        self.bias = bias
    def bewerte(self, antrag):
        pos = 50
        anteil = 45/self.regeln_pos.__len__()
        for key in self.regeln_pos.keys():
            if(self.regeln_pos[key].__contains__(antrag[key])):
                pos += anteil
            else:
                pos -= anteil
        try:
            pos = pos/100
            neg = 1-pos
            antrag["Bewertung"] = np.random.choice(["positiv", "negativ"], p=[pos, neg])
        except:
            print("SHIT")
        return antrag

In [16]:
class Bewerter:
    def __init__(self, regeln_pos, regeln_neg, bias):
        self.regeln_pos = regeln_pos
        self.regeln_neg = regeln_neg
        self.bias = bias
    def bewerte(self, antrag, bias):
        pos = 50
        anteil = 45/self.regeln_pos.__len__()
        for key in self.regeln_pos.keys():
            if(self.regeln_pos[key].__contains__(antrag[key])):
                pos += anteil
            else:
                pos -= anteil
        try:
            if(self.bias):
                for b in bias:
                    if(antrag[b]==bias[b]):
                        pos -= pos*0.8
            pos = pos/100
            neg = 1-pos
            antrag["Bewertung"] = np.random.choice(["positiv", "negativ"], p=[pos, neg])
        except:
            print("SHIT")
        return antrag

In [17]:
def work(df, bias, anzahlBewerter):
    antrag_füllwerte = {
        "Name": "random"
    }
    antrag_werte = {
        "Laufende_Strafe": [1,2,3,4,5],
        "Haerte_des_Vergehens": ["Leicht", "Mittel", "Hart"]
    }
    antrag_bias = {
        "Hautfarbe": ["Black", "White"],
        "Geschlecht": ["M", "W"]
    }
    r_pos, r_neg, rb_pos, rb_neg = regeln_erstellen(antrag_werte, antrag_bias, bias)
    #Bewerten
    bewerter = []
    for x in range(anzahlBewerter):
        bewerter.append(Bewerter(regeln_pos=r_pos, regeln_neg=r_neg, bias=False))
    biasBewerter = round(anzahlBewerter * 0.5)
    if(biasBewerter == 0):
        biasBewerter = 1
    for x in range(biasBewerter):
        bewerter[x].regeln_pos = rb_pos
        bewerter[x].regeln_neg = rb_neg
        bewerter[x].bias = True
    i = 0
    fertige_antraege = []
    for index, r in df.iterrows():
        #if(i >= anzahlBewerter):
        #    i = 0
        i = random.randint(0, anzahlBewerter-1)
        antrag = bewerter[i].bewerte(r.copy(), bias)
        fertige_antraege.append(antrag)
        #i += 1
    newdf = pd.DataFrame(fertige_antraege)
    return newdf
    
def daten_generieren(anzahlDaten):
    seed = seed_generieren()
    df = pd.DataFrame(erstelle_fake_data(anzahlDaten,int(seed)))
    return df

In [18]:
bias = {
    "Hautfarbe": ["Black"]
}
data = daten_generieren(1000)
#print(data)
finished = work(data,bias,5)
#print(finished)
data.to_csv("Daten.csv", sep=';', encoding='utf-8', index=False)
finished.to_csv("Daten_Bewertet.csv", sep=';', encoding='utf-8', index=False)

52.95043749407907
